# 🎬 Movies Big Data Pipeline - Google Colab

Pipeline completo de transformação de dados do dataset "The Movies Dataset" do Kaggle.

**Etapas:**
1. ⚙️ Configuração e Instalação
2. 📥 Ingestion (Bronze Layer)
3. 🔄 Transformation (Silver Layer)
4. 📊 Analytics Loading (Gold Layer)
5. 📈 Visualizações

**Dataset:** [The Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset)


## Resumo das Transformações

### Bronze → Silver Layer

#### 1. **Movies (movies_metadata.csv → movies.parquet)**
- **Parse de JSON**: genres, production_companies, countries, languages, collection
- **Conversão de tipos**: budget, revenue, runtime, ratings para numéricos
- **Extração de ano**: release_year do release_date
- **Cálculos derivados**: profit, ROI, flags has_budget/has_revenue
- **Filtros**: apenas filmes com status='Released' e ano válido
- **Resultado**: 45.379 filmes válidos, 30 colunas estruturadas

#### 2. **Credits (credits.csv → credits.parquet)**
- **Parse de cast**: extrair top 5 atores por ordem de aparição
- **Parse de crew**: extrair diretor (job='Director')
- **Resultado**: 45.476 registros, 5 colunas (cast, crew, director)

#### 3. **Keywords (keywords.csv → keywords.parquet)**
- **Parse de keywords**: converter JSON em lista de strings
- **Resultado**: 46.419 registros, 3 colunas (keyword_ids, keyword_names)

#### 4. **Ratings (ratings_small.csv → ratings.parquet)**
- **Conversão de timestamp**: Unix timestamp → datetime
- **Resultado**: 100.004 avaliações válidas

### Silver → Gold Layer

#### 1. **Movies Enriched (dataset completo)**
- **Merge**: movies + credits + keywords
- **Resultado**: 46.543 filmes com todas as informações

#### 2. **Yearly Analytics (por ano)**
- **Agregações**: COUNT, AVG, SUM por release_year
- **Métricas**: movie_count, budget, revenue, profit, rating, popularity, runtime
- **Resultado**: 120 anos analisados (1900-2020)

#### 3. **Genre Analytics (por gênero)**
- **Explode**: um filme pode ter múltiplos gêneros
- **Agregações**: COUNT, AVG, SUM por gênero
- **Resultado**: 20 gêneros com estatísticas completas

#### 4. **Top Movies (rankings)**
- **Top 100 por receita**
- **Top 100 por lucro**
- **Top 100 por avaliação** (mínimo 100 votos)
- **Resultado**: 300 filmes ranqueados

#### 5. **Director Analytics (por diretor)**
- **Agregações**: COUNT, AVG, MAX, SUM por diretor
- **Filtro**: apenas diretores com 2+ filmes
- **Resultado**: 4.351 diretores analisados

---

### Resumo de Tamanho

| Camada | Arquivos | Tamanho | Compressão |
|--------|----------|---------|------------|
| **Bronze** | 7 CSVs | ~900 MB | - |
| **Silver** | 4 Parquets | ~50 MB | **94%** |
| **Gold** | 5 Parquets | ~22 MB | **98%** |

**Total de redução:** 900 MB → 72 MB


## Fluxograma do Pipeline

```mermaid
flowchart TB
    %% Fonte
    kaggle[(Kaggle Dataset)]
    
    %% Bronze Layer
    subgraph bronze[" BRONZE LAYER "]
        raw[4 arquivos CSV<br/>900 MB]
    end
    
    %% Silver Layer
    subgraph silver[" SILVER LAYER "]
        proc[Parse JSON + Conversão + Filtros]
        silver_files[4 arquivos Parquet<br/>50 MB]
        proc --> silver_files
    end
    
    %% Gold Layer
    subgraph gold[" GOLD LAYER "]
        merge[Merge + Agregações]
        gold_files[5 arquivos Parquet<br/>22 MB]
        merge --> gold_files
    end
    
    %% Visualizações
    viz[Visualizações Plotly<br/>7 gráficos interativos]
    
    %% Fluxo
    kaggle -->|Download| bronze
    bronze -->|ETL| silver
    silver -->|Analytics| gold
    gold -->|Plotly| viz
    
    %% Estilos
    classDef bronzeStyle fill:#8B4513,stroke:#654321,stroke-width:2px,color:#000
    classDef silverStyle fill:#C0C0C0,stroke:#808080,stroke-width:2px,color:#000
    classDef goldStyle fill:#FFD700,stroke:#DAA520,stroke-width:2px,color:#000
    classDef vizStyle fill:#87CEEB,stroke:#4682B4,stroke-width:2px,color:#000
    
    class bronze,raw bronzeStyle
    class silver,proc,silver_files silverStyle
    class gold,merge,gold_files goldStyle
    class viz vizStyle
```

**Resumo do Pipeline:**
- **Bronze**: Ingestão de dados brutos (CSV, 900 MB)
- **Silver**: Transformação e limpeza (Parquet, 50 MB - 94% redução)
- **Gold**: Agregações analíticas (Parquet, 22 MB - 98% redução)
- **Visualizações**: 7 gráficos interativos com Plotly


## 1️⃣ Configuração Inicial

Instalação de dependências e setup do ambiente.


In [ ]:
# Instalar dependências
!pip install -q kaggle pandas pyarrow plotly


In [ ]:
# Imports
import os
import json
import zipfile
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files

print("✅ Bibliotecas importadas com sucesso!")


### 🔑 Configurar Credenciais Kaggle

**IMPORTANTE:** Faça upload do seu arquivo `kaggle.json`:
1. Baixe de: https://www.kaggle.com/settings
2. Execute a célula abaixo e faça upload do arquivo


In [ ]:
# Upload do kaggle.json
print("📤 Faça upload do arquivo kaggle.json:")
uploaded = files.upload()

# Configurar credenciais Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Credenciais Kaggle configuradas!")


In [ ]:
# Criar estrutura de diretórios
!mkdir -p data/raw data/processed data/refined

print("✅ Estrutura de diretórios criada!")


## 2️⃣ Ingestion - Bronze Layer

Download dos dados brutos do Kaggle.


In [ ]:
# Download do dataset
print("📥 Baixando dataset do Kaggle...")
!kaggle datasets download -d rounakbanik/the-movies-dataset -p data/raw --unzip

print("✅ Dataset baixado com sucesso!")

# Listar arquivos baixados
files_list = os.listdir('data/raw')
print(f"\n📁 Arquivos baixados ({len(files_list)}):")
for f in files_list:
    size_mb = os.path.getsize(f'data/raw/{f}') / (1024 * 1024)
    print(f"  - {f} ({size_mb:.2f} MB)")


## 3️⃣ Transformation - Silver Layer

Limpeza, validação e estruturação dos dados.


### 3.1 Transformação de Filmes


In [ ]:
print("🔄 Transformando dados de filmes...")


df_movies = pd.read_csv('data/raw/movies_metadata.csv', low_memory=False)
print(f"📊 Filmes carregados: {len(df_movies)}")


def safe_json_parse(json_str):
    try:
        if pd.isna(json_str):
            return []
        return json.loads(json_str.replace("'", '"'))
    except:
        return []


print("  Parsing genres...")
df_movies['genres_parsed'] = df_movies['genres'].apply(safe_json_parse)
df_movies['genre_ids'] = df_movies['genres_parsed'].apply(
    lambda x: ','.join([str(g['id']) for g in x]) if x else ''
)
df_movies['genre_names'] = df_movies['genres_parsed'].apply(
    lambda x: ', '.join([g['name'] for g in x]) if x else ''
)

print("  Parsing production companies...")
df_movies['companies_parsed'] = df_movies['production_companies'].apply(safe_json_parse)
df_movies['company_ids'] = df_movies['companies_parsed'].apply(
    lambda x: ','.join([str(c['id']) for c in x]) if x else ''
)
df_movies['company_names'] = df_movies['companies_parsed'].apply(
    lambda x: ', '.join([c['name'] for c in x]) if x else ''
)

print("  Parsing production countries...")
df_movies['countries_parsed'] = df_movies['production_countries'].apply(safe_json_parse)
df_movies['country_codes'] = df_movies['countries_parsed'].apply(
    lambda x: ','.join([c['iso_3166_1'] for c in x]) if x else ''
)
df_movies['country_names'] = df_movies['countries_parsed'].apply(
    lambda x: ', '.join([c['name'] for c in x]) if x else ''
)

print("  Parsing spoken languages...")
df_movies['languages_parsed'] = df_movies['spoken_languages'].apply(safe_json_parse)
df_movies['language_codes'] = df_movies['languages_parsed'].apply(
    lambda x: ','.join([l['iso_639_1'] for l in x]) if x else ''
)

print("  Parsing collection...")
df_movies['collection_parsed'] = df_movies['belongs_to_collection'].apply(safe_json_parse)
df_movies['collection_id'] = df_movies['collection_parsed'].apply(
    lambda x: x.get('id') if isinstance(x, dict) else None
)
df_movies['collection_name'] = df_movies['collection_parsed'].apply(
    lambda x: x.get('name') if isinstance(x, dict) else None
)


print("  Convertendo tipos de dados...")
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce').fillna(0)
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce').fillna(0)
df_movies['runtime'] = pd.to_numeric(df_movies['runtime'], errors='coerce')
df_movies['vote_average'] = pd.to_numeric(df_movies['vote_average'], errors='coerce').fillna(0)
df_movies['vote_count'] = pd.to_numeric(df_movies['vote_count'], errors='coerce').fillna(0).astype(int)
df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce').fillna(0)


df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')
df_movies['release_year'] = df_movies['release_date'].dt.year


df_movies['profit'] = df_movies['revenue'] - df_movies['budget']
df_movies['roi'] = ((df_movies['revenue'] - df_movies['budget']) / df_movies['budget'] * 100).replace([np.inf, -np.inf], 0).fillna(0)
df_movies['has_budget'] = df_movies['budget'] > 0
df_movies['has_revenue'] = df_movies['revenue'] > 0


print("  Filtrando dados válidos...")
df_movies_clean = df_movies[
    (df_movies['status'] == 'Released') &
    (df_movies['release_year'].notna())
].copy()


movies_columns = [
    'id', 'title', 'release_year', 'release_date', 'budget', 'revenue', 'profit', 'roi',
    'runtime', 'vote_average', 'vote_count', 'popularity', 'status', 'original_language',
    'collection_id', 'collection_name', 'genre_ids', 'genre_names',
    'company_ids', 'company_names', 'country_codes', 'country_names',
    'language_codes', 'has_budget', 'has_revenue', 'homepage', 'tagline', 'overview',
    'adult', 'video'
]

df_movies_final = df_movies_clean[movies_columns].copy()
df_movies_final.rename(columns={'id': 'movie_id'}, inplace=True)


df_movies_final.to_parquet('data/processed/movies.parquet', index=False)

print(f"✅ Filmes transformados: {len(df_movies_final)} de {len(df_movies)}")
print(f"   Colunas: {len(df_movies_final.columns)}")


### 3.2 Transformação de Créditos


In [ ]:
print("🔄 Transformando créditos...")


df_credits = pd.read_csv('data/raw/credits.csv')
print(f"📊 Créditos carregados: {len(df_credits)}")


print("  Parsing cast...")
df_credits['cast_parsed'] = df_credits['cast'].apply(safe_json_parse)
df_credits['cast_ids'] = df_credits['cast_parsed'].apply(
    lambda x: ','.join([str(c['id']) for c in x[:5]]) if x else ''
)
df_credits['cast_names'] = df_credits['cast_parsed'].apply(
    lambda x: ', '.join([c['name'] for c in x[:5]]) if x else ''
)


print("  Parsing crew e extraindo diretor...")
df_credits['crew_parsed'] = df_credits['crew'].apply(safe_json_parse)
df_credits['crew_ids'] = df_credits['crew_parsed'].apply(
    lambda x: ','.join([str(c['id']) for c in x[:10]]) if x else ''
)

def extract_director(crew_list):
    if not crew_list:
        return ''
    directors = [c['name'] for c in crew_list if c.get('job') == 'Director']
    return directors[0] if directors else ''

df_credits['director'] = df_credits['crew_parsed'].apply(extract_director)


df_credits_final = df_credits[['id', 'cast_ids', 'cast_names', 'crew_ids', 'director']].copy()
df_credits_final.rename(columns={'id': 'movie_id'}, inplace=True)


df_credits_final.to_parquet('data/processed/credits.parquet', index=False)

print(f"✅ Créditos transformados: {len(df_credits_final)}")


### 3.3 Transformação de Keywords


In [ ]:
print("🔄 Transformando keywords...")


df_keywords = pd.read_csv('data/raw/keywords.csv')
print(f"📊 Keywords carregadas: {len(df_keywords)}")


df_keywords['keywords_parsed'] = df_keywords['keywords'].apply(safe_json_parse)
df_keywords['keyword_ids'] = df_keywords['keywords_parsed'].apply(
    lambda x: ','.join([str(k['id']) for k in x]) if x else ''
)
df_keywords['keyword_names'] = df_keywords['keywords_parsed'].apply(
    lambda x: ', '.join([k['name'] for k in x]) if x else ''
)


df_keywords_final = df_keywords[['id', 'keyword_ids', 'keyword_names']].copy()
df_keywords_final.rename(columns={'id': 'movie_id'}, inplace=True)


df_keywords_final.to_parquet('data/processed/keywords.parquet', index=False)

print(f"✅ Keywords transformadas: {len(df_keywords_final)}")


In [ ]:
print("🔄 Transformando ratings...")

# Carregar ratings_small.csv
df_ratings = pd.read_csv('data/raw/ratings_small.csv')
print(f"📊 Ratings carregados: {len(df_ratings)}")

# Converter timestamp
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'], unit='s')

# Salvar em Parquet
df_ratings.to_parquet('data/processed/ratings.parquet', index=False)

print(f"✅ Ratings transformados: {len(df_ratings)}")


## 4️⃣ Analytics Loading - Gold Layer

Criação de datasets agregados para análise.


In [ ]:
print("📊 Carregando dados processados...")

# Carregar dados da Silver Layer
movies = pd.read_parquet('data/processed/movies.parquet')
credits = pd.read_parquet('data/processed/credits.parquet')
keywords = pd.read_parquet('data/processed/keywords.parquet')

print(f"  ✅ Movies: {len(movies)}")
print(f"  ✅ Credits: {len(credits)}")
print(f"  ✅ Keywords: {len(keywords)}")


### 4.1 Movies Enriched (Dataset Completo)


In [ ]:
print("🔗 Criando dataset enriquecido...")

# Merge de todos os dados
movies_enriched = movies.merge(credits, on='movie_id', how='left')
movies_enriched = movies_enriched.merge(keywords, on='movie_id', how='left')

# Salvar
movies_enriched.to_parquet('data/refined/movies_enriched.parquet', index=False)

print(f"✅ Movies Enriched: {len(movies_enriched)} filmes, {len(movies_enriched.columns)} colunas")


### 4.2 Yearly Analytics


In [ ]:
print("📅 Gerando analytics por ano...")

yearly_analytics = movies_enriched.groupby('release_year').agg({
    'movie_id': 'count',
    'budget': ['mean', 'sum'],
    'revenue': ['mean', 'sum'],
    'profit': ['mean', 'sum'],
    'vote_average': 'mean',
    'popularity': 'mean',
    'runtime': 'mean'
}).reset_index()

# Flatten multi-level columns
yearly_analytics.columns = [
    'release_year', 'movie_count',
    'avg_budget', 'total_budget',
    'avg_revenue', 'total_revenue',
    'avg_profit', 'total_profit',
    'avg_rating', 'avg_popularity', 'avg_runtime'
]

# Salvar
yearly_analytics.to_parquet('data/refined/yearly_analytics.parquet', index=False)

print(f"✅ Yearly Analytics: {len(yearly_analytics)} anos")


### 4.3 Genre Analytics


In [ ]:
print("🎭 Gerando analytics por gênero...")

# Explodir gêneros (um filme pode ter múltiplos gêneros)
movies_with_genres = movies_enriched[movies_enriched['genre_names'].notna()].copy()
movies_with_genres['genre_list'] = movies_with_genres['genre_names'].str.split(', ')
movies_genres_exploded = movies_with_genres.explode('genre_list')

# Agregações por gênero
genre_analytics = movies_genres_exploded.groupby('genre_list').agg({
    'movie_id': 'count',
    'budget': 'mean',
    'revenue': ['mean', 'sum'],
    'profit': 'mean',
    'vote_average': 'mean',
    'popularity': 'mean',
    'roi': 'mean'
}).reset_index()

# Flatten columns
genre_analytics.columns = [
    'genre_names', 'movie_count', 'avg_budget',
    'avg_revenue', 'total_revenue', 'avg_profit',
    'avg_rating', 'avg_popularity', 'avg_roi'
]

# Ordenar por número de filmes
genre_analytics = genre_analytics.sort_values('movie_count', ascending=False)

# Salvar
genre_analytics.to_parquet('data/refined/genre_analytics.parquet', index=False)

print(f"✅ Genre Analytics: {len(genre_analytics)} gêneros")


### 4.4 Top Movies


In [ ]:
print("🏆 Gerando top movies...")

# Top 100 por receita
top_revenue = movies_enriched.nlargest(100, 'revenue').copy()
top_revenue['rank'] = range(1, 101)
top_revenue['rank_type'] = 'revenue'

# Top 100 por lucro
top_profit = movies_enriched.nlargest(100, 'profit').copy()
top_profit['rank'] = range(1, 101)
top_profit['rank_type'] = 'profit'

# Top 100 por avaliação
top_rating = movies_enriched[movies_enriched['vote_count'] >= 100].nlargest(100, 'vote_average').copy()
top_rating['rank'] = range(1, 101)
top_rating['rank_type'] = 'rating'

# Concatenar
top_movies = pd.concat([top_revenue, top_profit, top_rating], ignore_index=True)

# Selecionar colunas relevantes
top_movies_cols = [
    'rank', 'rank_type', 'movie_id', 'title', 'release_year',
    'revenue', 'profit', 'roi', 'vote_average', 'vote_count',
    'genre_names', 'director', 'runtime'
]
top_movies = top_movies[top_movies_cols]

# Salvar
top_movies.to_parquet('data/refined/top_movies.parquet', index=False)

print(f"✅ Top Movies: {len(top_movies)} filmes")


### 4.5 Director Analytics


In [ ]:
print("🎬 Gerando analytics por diretor...")

# Filtrar filmes com diretor
movies_with_director = movies_enriched[
    (movies_enriched['director'].notna()) & 
    (movies_enriched['director'] != '')
].copy()

# Agregações por diretor
director_analytics = movies_with_director.groupby('director').agg({
    'movie_id': 'count',
    'budget': 'mean',
    'revenue': ['mean', 'sum'],
    'profit': 'mean',
    'vote_average': ['mean', 'max'],
    'popularity': 'mean'
}).reset_index()

# Flatten columns
director_analytics.columns = [
    'director', 'movie_count', 'avg_budget',
    'avg_revenue', 'total_revenue', 'avg_profit',
    'avg_rating', 'best_rating', 'avg_popularity'
]

# Filtrar diretores com pelo menos 2 filmes
director_analytics = director_analytics[director_analytics['movie_count'] >= 2]

# Ordenar por receita total
director_analytics = director_analytics.sort_values('total_revenue', ascending=False)

# Salvar
director_analytics.to_parquet('data/refined/director_analytics.parquet', index=False)

print(f"✅ Director Analytics: {len(director_analytics)} diretores")


## 5️⃣ Visualizações

Gráficos interativos dos dados processados.


### 📈 Gráfico 1: Evolução da Produção de Filmes


In [ ]:
# Filtrar dados de 1990 em diante
yearly_recent = yearly_analytics[yearly_analytics['release_year'] >= 1990]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yearly_recent['release_year'],
    y=yearly_recent['movie_count'],
    mode='lines+markers',
    name='Filmes Produzidos',
    line=dict(color='#1f77b4', width=3),
    fill='tozeroy',
    fillcolor='rgba(31, 119, 180, 0.2)'
))

fig.update_layout(
    title='📅 Produção de Filmes por Ano (1990+)',
    xaxis_title='Ano',
    yaxis_title='Número de Filmes',
    hovermode='x unified',
    height=500
)

fig.show()


### 💰 Gráfico 2: Evolução de Receita (Dual Axis)


In [ ]:
fig = go.Figure()

# Receita total (barras)
fig.add_trace(go.Bar(
    x=yearly_recent['release_year'],
    y=yearly_recent['total_revenue'] / 1e9,
    name='Receita Total',
    marker_color='#2ecc71',
    opacity=0.7,
    yaxis='y'
))

# Receita média (linha)
fig.add_trace(go.Scatter(
    x=yearly_recent['release_year'],
    y=yearly_recent['avg_revenue'] / 1e6,
    name='Receita Média/Filme',
    yaxis='y2',
    line=dict(color='#e74c3c', width=3),
    mode='lines+markers'
))

fig.update_layout(
    title='💰 Evolução de Receita (1990+)',
    xaxis_title='Ano',
    yaxis=dict(title='Receita Total (Bilhões $)'),
    yaxis2=dict(
        title='Receita Média (Milhões $)',
        overlaying='y',
        side='right'
    ),
    hovermode='x unified',
    height=500,
    legend=dict(x=0.01, y=0.99)
)

fig.show()


### 🎭 Gráfico 3: Top 10 Gêneros


In [ ]:
top_10_genres = genre_analytics.head(10)

fig = px.bar(
    top_10_genres,
    y='genre_names',
    x='movie_count',
    orientation='h',
    title='🎭 Top 10 Gêneros por Número de Filmes',
    labels={'movie_count': 'Número de Filmes', 'genre_names': 'Gênero'},
    color='movie_count',
    color_continuous_scale='Blues',
    height=500
)

fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()


### 💵 Gráfico 4: Receita Média por Gênero


In [ ]:
top_revenue_genres = genre_analytics.nlargest(10, 'avg_revenue')

fig = px.bar(
    top_revenue_genres,
    y='genre_names',
    x='avg_revenue',
    orientation='h',
    title='💵 Top 10 Gêneros por Receita Média',
    labels={'avg_revenue': 'Receita Média ($)', 'genre_names': 'Gênero'},
    color='avg_revenue',
    color_continuous_scale='Greens',
    height=500
)

fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.update_traces(
    hovertemplate='<b>%{y}</b><br>Receita: $%{x:,.0f}<extra></extra>'
)
fig.show()


### 🎬 Gráfico 5: Top 10 Diretores por Receita


In [ ]:
top_directors = director_analytics.head(10)

fig = px.bar(
    top_directors,
    y='director',
    x='total_revenue',
    orientation='h',
    title='🎬 Top 10 Diretores por Receita Total',
    labels={'total_revenue': 'Receita Total ($)', 'director': 'Diretor'},
    color='total_revenue',
    color_continuous_scale='Purples',
    height=500
)

fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.update_traces(
    hovertemplate='<b>%{y}</b><br>Receita: $%{x:,.0f}<extra></extra>'
)
fig.show()


### 💎 Gráfico 6: Scatter Plot - Orçamento vs Receita


In [ ]:
# Filtrar filmes com budget e revenue válidos
movies_scatter = movies_enriched[
    (movies_enriched['has_budget']) & 
    (movies_enriched['has_revenue']) &
    (movies_enriched['budget'] > 0)
].head(500)

fig = go.Figure()

# Scatter plot
fig.add_trace(go.Scatter(
    x=movies_scatter['budget'] / 1e6,
    y=movies_scatter['revenue'] / 1e6,
    mode='markers',
    marker=dict(
        size=8,
        color=movies_scatter['vote_average'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="Nota"),
        line=dict(width=0.5, color='white')
    ),
    text=movies_scatter['title'],
    hovertemplate='<b>%{text}</b><br>Orçamento: $%{x:.1f}M<br>Receita: $%{y:.1f}M<extra></extra>',
    showlegend=False
))

# Linha ROI = 100%
max_val = max(movies_scatter['budget'].max(), movies_scatter['revenue'].max()) / 1e6
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    line=dict(color='red', dash='dash', width=2),
    name='ROI = 100%',
    hoverinfo='skip'
))

fig.update_layout(
    title='💎 Orçamento vs Receita (Top 500 filmes)',
    xaxis_title='Orçamento (Milhões $)',
    yaxis_title='Receita (Milhões $)',
    height=600
)

fig.show()


### 🏆 Gráfico 7: Top 20 Filmes por Receita


In [ ]:
top_20_revenue = top_movies[top_movies['rank_type'] == 'revenue'].head(20)

fig = px.bar(
    top_20_revenue,
    y='title',
    x='revenue',
    orientation='h',
    title='🏆 Top 20 Filmes por Receita',
    labels={'revenue': 'Receita ($)', 'title': 'Filme'},
    color='revenue',
    color_continuous_scale='Viridis',
    height=700
)

fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.update_traces(
    hovertemplate='<b>%{y}</b><br>Receita: $%{x:,.0f}<br><extra></extra>'
)
fig.show()


## ✅ Resumo Final

Pipeline executado com sucesso!


In [ ]:
print("=" * 80)
print("🎬 MOVIES BIG DATA PIPELINE - RESUMO FINAL")
print("=" * 80)

print("\n📊 CAMADAS DE DADOS:")
print(f"  Bronze (Raw):       {len(os.listdir('data/raw'))} arquivos CSV")
print(f"  Silver (Processed): {len(os.listdir('data/processed'))} arquivos Parquet")
print(f"  Gold (Refined):     {len(os.listdir('data/refined'))} arquivos Parquet")

print("\n📈 ESTATÍSTICAS:")
print(f"  Total de Filmes:    {len(movies_enriched):,}")
print(f"  Período:            {int(movies_enriched['release_year'].min())} - {int(movies_enriched['release_year'].max())}")
print(f"  Gêneros:            {len(genre_analytics)}")
print(f"  Diretores:          {len(director_analytics):,}")
print(f"  Anos Analisados:    {len(yearly_analytics)}")

movies_with_finance = movies_enriched[movies_enriched['has_budget'] & movies_enriched['has_revenue']]
print(f"\n💰 FINANÇAS:")
print(f"  Receita Média:      ${movies_with_finance['revenue'].mean()/1e6:.1f}M")
print(f"  Receita Total:      ${movies_with_finance['revenue'].sum()/1e9:.1f}B")
print(f"  Orçamento Médio:    ${movies_with_finance['budget'].mean()/1e6:.1f}M")
print(f"  ROI Médio:          {movies_with_finance['roi'].mean():.1f}%")

print(f"\n⭐ AVALIAÇÕES:")
rated_movies = movies_enriched[movies_enriched['vote_average'] > 0]
print(f"  Nota Média:         {rated_movies['vote_average'].mean():.2f}/10")
print(f"  Filmes Avaliados:   {len(rated_movies):,}")

print(f"\n🏆 TOP 3:")
print(f"\n  Gêneros mais populares:")
for i, row in genre_analytics.head(3).iterrows():
    print(f"    {i+1}. {row['genre_names']}: {int(row['movie_count']):,} filmes")

print(f"\n  Diretores mais lucrativos:")
for i, row in director_analytics.head(3).iterrows():
    print(f"    {i+1}. {row['director']}: ${row['total_revenue']/1e9:.2f}B")

print(f"\n  Filmes de maior receita:")
for i, row in top_revenue.head(3).iterrows():
    print(f"    {i+1}. {row['title']} ({int(row['release_year'])}): ${row['revenue']/1e9:.2f}B")

print("\n" + "=" * 80)
print("✅ PIPELINE COMPLETO EXECUTADO COM SUCESSO!")
print("=" * 80)
